In [2]:
print('Fuck Yeah!')

Fuck Yeah!


In [3]:
print('we are goingto make more changes')

we are goingto make more changes


In [1]:
print('we made changes and it going to git hub!')

we made changes and it going to git hub!
